<a href="https://colab.research.google.com/github/jorge-flores-py/PROCESAMIENTO-DE-HABLA-IAPH/blob/main/Desafio_3_jorge_flores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
cossim

array([0.1382319 , 0.1067036 , 0.23029327, ..., 0.12320753, 0.08765353,
       0.04415046])

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 2

Alumno: Jorge Flores

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.




**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.



> Añadir blockquote



**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [30]:
import random
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# Cargar datos
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))


In [31]:
# Vectorizamos los documentos con TF-IDF
# - stop_words='english': elimina palabras comunes en inglés
# - max_df=0.7: elimina palabras que aparecen en más del 70% de los documentos
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=5, ngram_range=(1,2))

# Aplicamos la transformación a los textos de entrenamiento y test
X_train = vectorizer.fit_transform(train.data)
X_test = vectorizer.transform(test.data)

# Extraemos las etiquetas (clases) correspondientes
y_train = train.target
y_test = test.target


In [32]:
# Semilla para reproducibilidad
random.seed(42)

# Seleccionamos 5 documentos al azar del conjunto de entrenamiento
indices_azar = random.sample(range(len(train.data)), 5)

# Recorremos cada documento seleccionado
for idx in indices_azar:
    print("=" * 100)
    print(f"DOCUMENTO ORIGINAL - Índice: {idx}")
    print("-" * 100)

    # Mostramos parte del texto original, formateado en líneas de ancho 100
    texto_original = train.data[idx][:500].strip()
    print(textwrap.fill(texto_original, width=100))

    # Mostramos la clase del documento original
    print(f"\nClase: {train.target_names[y_train[idx]]}")
    print("=" * 100)

    # Calculamos la similitud coseno con todos los documentos del entrenamiento
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Excluimos el propio documento
    cossim[idx] = 0

    # Obtenemos el índice del documento más similar
    idx_mas_similar = np.argmax(cossim)

    # Mostramos la información del documento más similar
    print("DOCUMENTO MÁS SIMILAR:")
    print("-" * 100)
    print(f"Índice: {idx_mas_similar}")
    print(f"Clase: {train.target_names[y_train[idx_mas_similar]]}")
    print(f"Similitud coseno: {cossim[idx_mas_similar]:.4f}")

    # Mostramos parte del texto del documento más similar
    texto_similar = train.data[idx_mas_similar][:500].strip()
    print(textwrap.fill(texto_similar, width=100))
    print("=" * 100)
    print("\n\n")


DOCUMENTO ORIGINAL - Índice: 10476
----------------------------------------------------------------------------------------------------
This is a general question for US readers:  How extensive is the playoff coverage down there?  In
Canada, it is almost impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because I'm in the wrong time
zone!).  We (in Canada) are basically swamped with  coverage, and I wonder how many series/games are
televised nationally or even locally in the US and how much precedence they take over, say, local
new

Clase: rec.sport.hockey
DOCUMENTO MÁS SIMILAR:
----------------------------------------------------------------------------------------------------
Índice: 5064
Clase: rec.sport.hockey
Similitud coseno: 0.1930
I only have one comment on this:  You call this a *classic* playoff year and yet you don't include a
Chicago-Detroit series.  C'mon, I'm a Boston fan an


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [20]:

# Instanciamos un modelo de clasificación Naïve Bayes Multinomial
clf = MultinomialNB()

# Entrenamos el modelo con los datos vectorizados de entrenamiento
clf.fit(X_train, y_train)

# Usamos el mismo vectorizador TF-IDF ya entrenado (fiteado) para transformar los textos del test
# Esto asegura que se usen las mismas palabras y pesos que en entrenamiento
X_test = vectorizer.transform(test.data)
y_test = test.target  # etiquetas verdaderas del conjunto de test

# Hacemos predicciones sobre los documentos del conjunto de test
y_pred = clf.predict(X_test)

# Calculamos el F1-score macro, que promedia el F1 de cada clase (sin importar cuántos ejemplos tenga cada clase)
# Es más robusto ante desbalance de clases que la accuracy
f1_macro = f1_score(y_test, y_pred, average='macro')

# Mostramos el resultado
print(f"F1-score macro (MultinomialNB): {f1_macro:.4f}")


F1-score macro (MultinomialNB): 0.7940





**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [29]:
# Transponemos la matriz documento-término para obtener término-documento
# Cada fila representa una palabra (vector de aparición en documentos)
term_doc_matrix = X_train.T  # (vocab_size, n_docs)

# Obtenemos el vocabulario como array indexado correctamente
vocabulario = vectorizer.get_feature_names_out()

# Creamos un diccionario de palabra → índice para buscar más fácilmente
word_to_index = {word: idx for idx, word in enumerate(vocabulario)}

# Elegimos 5 palabras manualmente (asegurar que sean interpretables y estén en el vocabulario)
palabras_elegidas = ["space", "jesus", "argentine", "universe", "satellite"]

# Recorremos cada palabra seleccionada
for palabra in palabras_elegidas:
    print("=" * 100)
    print(f"PALABRA BASE: '{palabra}'")

    # Verificamos que la palabra esté en el vocabulario
    if palabra not in word_to_index:
        print(" → Esta palabra no está en el vocabulario del vectorizador.")
        continue

    # Obtenemos el vector de la palabra (como aparece en todos los documentos)
    idx = word_to_index[palabra]
    vector_palabra = term_doc_matrix[idx]

    # Calculamos similitud coseno de esta palabra contra todas las demás
    similitudes = cosine_similarity(vector_palabra, term_doc_matrix).flatten()

    # Excluimos a sí misma para no aparecer como "más similar"
    similitudes[idx] = 0

    # Obtenemos las 5 palabras con mayor similitud
    top_similares = np.argsort(similitudes)[::-1][:5]

    # Mostramos las palabras más similares
    print("\nPalabras más similares:")
    for i in top_similares:
        palabra_similar = vocabulario[i]  # acceso correcto garantizado
        score = similitudes[i]
        print(f"- {palabra_similar:15s} (similitud: {score:.4f})")


PALABRA BASE: 'space'

Palabras más similares:
- shuttle         (similitud: 0.3854)
- space shuttle   (similitud: 0.3796)
- space station   (similitud: 0.3658)
- space digest    (similitud: 0.3536)
- subject space   (similitud: 0.3400)
PALABRA BASE: 'jesus'

Palabras más similares:
- christ          (similitud: 0.3714)
- did jesus       (similitud: 0.3291)
- god             (similitud: 0.3227)
- said jesus      (similitud: 0.3196)
- subject prayer  (similitud: 0.3191)
PALABRA BASE: 'argentine'
 → Esta palabra no está en el vocabulario del vectorizador.
PALABRA BASE: 'universe'

Palabras más similares:
- interact        (similitud: 0.4156)
- universe just   (similitud: 0.4036)
- does universe   (similitud: 0.3989)
- universe exist  (similitud: 0.3928)
- measurable      (similitud: 0.3692)
PALABRA BASE: 'satellite'

Palabras más similares:
- satellite launched (similitud: 0.5444)
- launched        (similitud: 0.4567)
- satellites      (similitud: 0.4123)
- itar            (similitud: 0.